In [2]:
import os 
import pandas as pd
import numpy as np
from pydub import AudioSegment

import IPython.display as ipd

import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt

import processing

/Users/bastienorset/Documents/git/respiratory_sound/.venv/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
foldername = '../../data/ICBHI_final_database'

In [4]:
import csv

file_to_write = 'dataset2.csv'
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 13):
    header += f' chroma_stft{i}'
for i in range(1, 21):
    header += f' mfcc{i}'
for i in range(1, 21):
    header += f' delta-mfcc{i}'
for i in range(1, 21):
    header += f' delta2-mfcc{i}'
header += ' label'
header = header.split()

file = open(file_to_write, 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    arrays = []
for file in os.listdir(foldername):
    if file.endswith('.wav'):
        wav_file = os.path.join(foldername,file)
        data = processing.extract_data_from_file(wav_file)
        data_filtered = processing.apply_filtering_on_signal(data, lf=120, hf= 1800, backward = True)
        rmse = librosa.feature.rms(y=data_filtered['data'])
        chroma_stft = librosa.feature.chroma_stft(y=data_filtered['data'], sr=data_filtered['fs'])
        spec_cent = librosa.feature.spectral_centroid(y=data_filtered['data'], sr=data_filtered['fs'])
        spec_bw = librosa.feature.spectral_bandwidth(y=data_filtered['data'], sr=data_filtered['fs'])
        rolloff = librosa.feature.spectral_rolloff(y=data_filtered['data'], sr=data_filtered['fs'])
        zcr = librosa.feature.zero_crossing_rate(data_filtered['data'])
        mfcc = librosa.feature.mfcc(y=data_filtered['data'], sr=data_filtered['fs'])
        delta_mfccs = librosa.feature.delta(mfcc)
        delta2_mfccs = librosa.feature.delta(mfcc,order=2)
        comprehensive_mfcc = np.concatenate((chroma_stft,mfcc,delta_mfccs,delta2_mfccs))
        to_append = f'{wav_file} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in comprehensive_mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {file}'
        file = open(file_to_write, 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

KeyboardInterrupt: 